In [ ]:
pip install sastrawi

     |████████████████████████████████| 209 kB 5.2 MB/s 


In [ ]:
pip install nltk

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import string
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AS/tweets50000.csv')

In [ ]:
df.shape

(50001, 3)

In [ ]:
#Cleaning
def remove_tweet_special(text):
    text = text.replace('\\t'," ").replace('\\n', " ").replace('\\u'," ").replace('\\'," ")
    text = text.encode('ascii','replace').decode('ascii')
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    return text.replace("http://", " ").replace("https://", " ")

df['Cleaning'] = df['Tweet'].apply(remove_tweet_special)
df.head()

,Datetime,Username,Tweet,Cleaning
0,2021-11-01 23:56:14+00:00,viligerrystory,ternyata kuliah msh online jg pdhl katanya off...,ternyata kuliah msh online jg pdhl katanya off...
1,2021-11-01 23:45:26+00:00,Fiayellowluv18,Gpp sihh hujan terus tapi jangan sama petir to...,Gpp sihh hujan terus tapi jangan sama petir to...
2,2021-11-01 23:18:49+00:00,sliwerr,"@ciebing sekolahh, tapi online hehe. kemarin g...","sekolahh, tapi online hehe. kemarin gimanaa ku..."
3,2021-11-01 23:08:21+00:00,deviiiayu96,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR? BIS...,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR? BIS...
4,2021-11-01 22:17:08+00:00,Adeliaprbw,Awal masuk poltekkes pasrah sadar diri ga masu...,Awal masuk poltekkes pasrah sadar diri ga masu...


In [ ]:
def remove_punct(text):
    text = re.sub(r'[^a-zA-z0-9]', ' ', str(text))
    text = re.sub(r'\b\w{1,2}\b', '', text) #menghilangkan 2 kata
    text = re.sub(r'\s\s+', ' ', text)
    return text
df['Cleaning'] = df['Cleaning'].apply(lambda x: remove_punct(x))
df.head()

,Datetime,Username,Tweet,Cleaning
0,2021-11-01 23:56:14+00:00,viligerrystory,ternyata kuliah msh online jg pdhl katanya off...,ternyata kuliah msh online pdhl katanya offline
1,2021-11-01 23:45:26+00:00,Fiayellowluv18,Gpp sihh hujan terus tapi jangan sama petir to...,Gpp sihh hujan terus tapi jangan sama petir to...
2,2021-11-01 23:18:49+00:00,sliwerr,"@ciebing sekolahh, tapi online hehe. kemarin g...",sekolahh tapi online hehe kemarin gimanaa kuli...
3,2021-11-01 23:08:21+00:00,deviiiayu96,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR? BIS...,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR BISA...
4,2021-11-01 22:17:08+00:00,Adeliaprbw,Awal masuk poltekkes pasrah sadar diri ga masu...,Awal masuk poltekkes pasrah sadar diri masuk b...


In [ ]:
def remove_number(text):
    return re.sub(r"\d+","", text)

df["Cleaning"] = df["Cleaning"].apply(remove_number)
df.head()

,Datetime,Username,Tweet,Cleaning
0,2021-11-01 23:56:14+00:00,viligerrystory,ternyata kuliah msh online jg pdhl katanya off...,ternyata kuliah msh online pdhl katanya offline
1,2021-11-01 23:45:26+00:00,Fiayellowluv18,Gpp sihh hujan terus tapi jangan sama petir to...,Gpp sihh hujan terus tapi jangan sama petir to...
2,2021-11-01 23:18:49+00:00,sliwerr,"@ciebing sekolahh, tapi online hehe. kemarin g...",sekolahh tapi online hehe kemarin gimanaa kuli...
3,2021-11-01 23:08:21+00:00,deviiiayu96,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR? BIS...,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR BISA...
4,2021-11-01 22:17:08+00:00,Adeliaprbw,Awal masuk poltekkes pasrah sadar diri ga masu...,Awal masuk poltekkes pasrah sadar diri masuk b...


In [ ]:
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b","", text)

df["Cleaning"] = df["Cleaning"].apply(remove_singl_char)
df.head()

,Datetime,Username,Tweet,Cleaning
0,2021-11-01 23:56:14+00:00,viligerrystory,ternyata kuliah msh online jg pdhl katanya off...,ternyata kuliah msh online pdhl katanya offline
1,2021-11-01 23:45:26+00:00,Fiayellowluv18,Gpp sihh hujan terus tapi jangan sama petir to...,Gpp sihh hujan terus tapi jangan sama petir to...
2,2021-11-01 23:18:49+00:00,sliwerr,"@ciebing sekolahh, tapi online hehe. kemarin g...",sekolahh tapi online hehe kemarin gimanaa kuli...
3,2021-11-01 23:08:21+00:00,deviiiayu96,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR? BIS...,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR BISA...
4,2021-11-01 22:17:08+00:00,Adeliaprbw,Awal masuk poltekkes pasrah sadar diri ga masu...,Awal masuk poltekkes pasrah sadar diri masuk b...


In [ ]:
#Case Folding
df['Case Folding'] = df['Cleaning'].str.lower()
df.head()

,Datetime,Username,Tweet,Cleaning,Case Folding
0,2021-11-01 23:56:14+00:00,viligerrystory,ternyata kuliah msh online jg pdhl katanya off...,ternyata kuliah msh online pdhl katanya offline,ternyata kuliah msh online pdhl katanya offline
1,2021-11-01 23:45:26+00:00,Fiayellowluv18,Gpp sihh hujan terus tapi jangan sama petir to...,Gpp sihh hujan terus tapi jangan sama petir to...,gpp sihh hujan terus tapi jangan sama petir to...
2,2021-11-01 23:18:49+00:00,sliwerr,"@ciebing sekolahh, tapi online hehe. kemarin g...",sekolahh tapi online hehe kemarin gimanaa kuli...,sekolahh tapi online hehe kemarin gimanaa kuli...
3,2021-11-01 23:08:21+00:00,deviiiayu96,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR? BIS...,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR BISA...,bisnis sambil sekolah kuliah atau ngantor bisa...
4,2021-11-01 22:17:08+00:00,Adeliaprbw,Awal masuk poltekkes pasrah sadar diri ga masu...,Awal masuk poltekkes pasrah sadar diri masuk b...,awal masuk poltekkes pasrah sadar diri masuk b...


In [ ]:
#Tokenization
def tokenization(text):
    text = re.split('\W+', text)
    return text
df['Tokenization'] = df['Case Folding'].apply(lambda x: tokenization(x.lower()))
df.head()

,Datetime,Username,Tweet,Cleaning,Case Folding,Tokenization
0,2021-11-01 23:56:14+00:00,viligerrystory,ternyata kuliah msh online jg pdhl katanya off...,ternyata kuliah msh online pdhl katanya offline,ternyata kuliah msh online pdhl katanya offline,"[ternyata, kuliah, msh, online, pdhl, katanya,..."
1,2021-11-01 23:45:26+00:00,Fiayellowluv18,Gpp sihh hujan terus tapi jangan sama petir to...,Gpp sihh hujan terus tapi jangan sama petir to...,gpp sihh hujan terus tapi jangan sama petir to...,"[gpp, sihh, hujan, terus, tapi, jangan, sama, ..."
2,2021-11-01 23:18:49+00:00,sliwerr,"@ciebing sekolahh, tapi online hehe. kemarin g...",sekolahh tapi online hehe kemarin gimanaa kuli...,sekolahh tapi online hehe kemarin gimanaa kuli...,"[sekolahh, tapi, online, hehe, kemarin, gimana..."
3,2021-11-01 23:08:21+00:00,deviiiayu96,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR? BIS...,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR BISA...,bisnis sambil sekolah kuliah atau ngantor bisa...,"[bisnis, sambil, sekolah, kuliah, atau, nganto..."
4,2021-11-01 22:17:08+00:00,Adeliaprbw,Awal masuk poltekkes pasrah sadar diri ga masu...,Awal masuk poltekkes pasrah sadar diri masuk b...,awal masuk poltekkes pasrah sadar diri masuk b...,"[awal, masuk, poltekkes, pasrah, sadar, diri, ..."


In [ ]:
#Normalisasi
normalizad_word = pd.read_csv('/content/drive/MyDrive/AS/colloquial-indonesian-lexicon.csv') 
normalizad_word_dict = {} 
for index, row in normalizad_word.iterrows(): 
    if row[0] not in normalizad_word_dict: 
        normalizad_word_dict[row[0]] = row[1] 
def normalized_term(document): 
  return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document] 
df['Normalisasi'] = df['Tokenization'].apply(normalized_term) 
df.head()

,Datetime,Username,Tweet,Cleaning,Case Folding,Tokenization,Normalisasi
0,2021-11-01 23:56:14+00:00,viligerrystory,ternyata kuliah msh online jg pdhl katanya off...,ternyata kuliah msh online pdhl katanya offline,ternyata kuliah msh online pdhl katanya offline,"[ternyata, kuliah, msh, online, pdhl, katanya,...","[ternyata, kuliah, masih, online, padahal, kat..."
1,2021-11-01 23:45:26+00:00,Fiayellowluv18,Gpp sihh hujan terus tapi jangan sama petir to...,Gpp sihh hujan terus tapi jangan sama petir to...,gpp sihh hujan terus tapi jangan sama petir to...,"[gpp, sihh, hujan, terus, tapi, jangan, sama, ...","[enggak apa-apa, sih, hujan, terus, tapi, jang..."
2,2021-11-01 23:18:49+00:00,sliwerr,"@ciebing sekolahh, tapi online hehe. kemarin g...",sekolahh tapi online hehe kemarin gimanaa kuli...,sekolahh tapi online hehe kemarin gimanaa kuli...,"[sekolahh, tapi, online, hehe, kemarin, gimana...","[sekolahh, tapi, online, hehe, kemarin, bagaim..."
3,2021-11-01 23:08:21+00:00,deviiiayu96,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR? BIS...,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR BISA...,bisnis sambil sekolah kuliah atau ngantor bisa...,"[bisnis, sambil, sekolah, kuliah, atau, nganto...","[bisnis, sambil, sekolah, kuliah, atau, nganto..."
4,2021-11-01 22:17:08+00:00,Adeliaprbw,Awal masuk poltekkes pasrah sadar diri ga masu...,Awal masuk poltekkes pasrah sadar diri masuk b...,awal masuk poltekkes pasrah sadar diri masuk b...,"[awal, masuk, poltekkes, pasrah, sadar, diri, ...","[awal, masuk, poltekkes, pasrah, sadar, diri, ..."


In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords

list_stopwords = stopwords.words('indonesian')
list_stopwords.extend(['loh', 'oiya', 'nih', 'ok', 'ah', 'nya', 'banget', 'gue', 'kalo', 'deh', 'pas','gua', 'gw', 'biar', 'iya', 'wkwk', 'hehe', 'kak', 'bisa'])
txt_stopword = pd.read_csv('/content/drive/MyDrive/AS/stopwords.txt', names= ["stopwords"], header = None)
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))
list_stopwords = set(list_stopwords)

def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords ]
df['SW Removal'] = df['Normalisasi'].apply(stopwords_removal)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Datetime,Username,Tweet,Cleaning,Case Folding,Tokenization,Normalisasi,SW Removal
0,2021-11-01 23:56:14+00:00,viligerrystory,ternyata kuliah msh online jg pdhl katanya off...,ternyata kuliah msh online pdhl katanya offline,ternyata kuliah msh online pdhl katanya offline,"[ternyata, kuliah, msh, online, pdhl, katanya,...","[ternyata, kuliah, masih, online, padahal, kat...","[kuliah, online, offline]"
1,2021-11-01 23:45:26+00:00,Fiayellowluv18,Gpp sihh hujan terus tapi jangan sama petir to...,Gpp sihh hujan terus tapi jangan sama petir to...,gpp sihh hujan terus tapi jangan sama petir to...,"[gpp, sihh, hujan, terus, tapi, jangan, sama, ...","[enggak apa-apa, sih, hujan, terus, tapi, jang...","[enggak apa-apa, sih, hujan, petir, tolong, ta..."
2,2021-11-01 23:18:49+00:00,sliwerr,"@ciebing sekolahh, tapi online hehe. kemarin g...",sekolahh tapi online hehe kemarin gimanaa kuli...,sekolahh tapi online hehe kemarin gimanaa kuli...,"[sekolahh, tapi, online, hehe, kemarin, gimana...","[sekolahh, tapi, online, hehe, kemarin, bagaim...","[sekolahh, online, kemarin, kuliah, offline, ]"
3,2021-11-01 23:08:21+00:00,deviiiayu96,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR? BIS...,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR BISA...,bisnis sambil sekolah kuliah atau ngantor bisa...,"[bisnis, sambil, sekolah, kuliah, atau, nganto...","[bisnis, sambil, sekolah, kuliah, atau, nganto...","[bisnis, sekolah, kuliah, ngantor, online, sho..."
4,2021-11-01 22:17:08+00:00,Adeliaprbw,Awal masuk poltekkes pasrah sadar diri ga masu...,Awal masuk poltekkes pasrah sadar diri masuk b...,awal masuk poltekkes pasrah sadar diri masuk b...,"[awal, masuk, poltekkes, pasrah, sadar, diri, ...","[awal, masuk, poltekkes, pasrah, sadar, diri, ...","[masuk, poltekkes, pasrah, sadar, masuk, disma..."


In [ ]:
import ast
def join_text_list(texts):
    return ' '.join([text for text in texts])

df['SW Removal'] = df['SW Removal'].apply(join_text_list)
df.head()

,Datetime,Username,Tweet,Cleaning,Case Folding,Tokenization,Normalisasi,SW Removal
0,2021-11-01 23:56:14+00:00,viligerrystory,ternyata kuliah msh online jg pdhl katanya off...,ternyata kuliah msh online pdhl katanya offline,ternyata kuliah msh online pdhl katanya offline,"[ternyata, kuliah, msh, online, pdhl, katanya,...","[ternyata, kuliah, masih, online, padahal, kat...",kuliah online offline
1,2021-11-01 23:45:26+00:00,Fiayellowluv18,Gpp sihh hujan terus tapi jangan sama petir to...,Gpp sihh hujan terus tapi jangan sama petir to...,gpp sihh hujan terus tapi jangan sama petir to...,"[gpp, sihh, hujan, terus, tapi, jangan, sama, ...","[enggak apa-apa, sih, hujan, terus, tapi, jang...",enggak apa-apa sih hujan petir tolong takut ka...
2,2021-11-01 23:18:49+00:00,sliwerr,"@ciebing sekolahh, tapi online hehe. kemarin g...",sekolahh tapi online hehe kemarin gimanaa kuli...,sekolahh tapi online hehe kemarin gimanaa kuli...,"[sekolahh, tapi, online, hehe, kemarin, gimana...","[sekolahh, tapi, online, hehe, kemarin, bagaim...",sekolahh online kemarin kuliah offline
3,2021-11-01 23:08:21+00:00,deviiiayu96,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR? BIS...,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR BISA...,bisnis sambil sekolah kuliah atau ngantor bisa...,"[bisnis, sambil, sekolah, kuliah, atau, nganto...","[bisnis, sambil, sekolah, kuliah, atau, nganto...",bisnis sekolah kuliah ngantor online shop prod...
4,2021-11-01 22:17:08+00:00,Adeliaprbw,Awal masuk poltekkes pasrah sadar diri ga masu...,Awal masuk poltekkes pasrah sadar diri masuk b...,awal masuk poltekkes pasrah sadar diri masuk b...,"[awal, masuk, poltekkes, pasrah, sadar, diri, ...","[awal, masuk, poltekkes, pasrah, sadar, diri, ...",masuk poltekkes pasrah sadar masuk disma ditri...


In [ ]:
#Tokenization SW
def tokenization(text):
    text = re.split('\W+', text)
    return text
df['SW Removal'] = df['SW Removal'].apply(lambda x: tokenization(x.lower()))
df.head()

,Datetime,Username,Tweet,Cleaning,Case Folding,Tokenization,Normalisasi,SW Removal
0,2021-11-01 23:56:14+00:00,viligerrystory,ternyata kuliah msh online jg pdhl katanya off...,ternyata kuliah msh online pdhl katanya offline,ternyata kuliah msh online pdhl katanya offline,"[ternyata, kuliah, msh, online, pdhl, katanya,...","[ternyata, kuliah, masih, online, padahal, kat...","[kuliah, online, offline]"
1,2021-11-01 23:45:26+00:00,Fiayellowluv18,Gpp sihh hujan terus tapi jangan sama petir to...,Gpp sihh hujan terus tapi jangan sama petir to...,gpp sihh hujan terus tapi jangan sama petir to...,"[gpp, sihh, hujan, terus, tapi, jangan, sama, ...","[enggak apa-apa, sih, hujan, terus, tapi, jang...","[enggak, apa, apa, sih, hujan, petir, tolong, ..."
2,2021-11-01 23:18:49+00:00,sliwerr,"@ciebing sekolahh, tapi online hehe. kemarin g...",sekolahh tapi online hehe kemarin gimanaa kuli...,sekolahh tapi online hehe kemarin gimanaa kuli...,"[sekolahh, tapi, online, hehe, kemarin, gimana...","[sekolahh, tapi, online, hehe, kemarin, bagaim...","[sekolahh, online, kemarin, kuliah, offline, ]"
3,2021-11-01 23:08:21+00:00,deviiiayu96,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR? BIS...,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR BISA...,bisnis sambil sekolah kuliah atau ngantor bisa...,"[bisnis, sambil, sekolah, kuliah, atau, nganto...","[bisnis, sambil, sekolah, kuliah, atau, nganto...","[bisnis, sekolah, kuliah, ngantor, online, sho..."
4,2021-11-01 22:17:08+00:00,Adeliaprbw,Awal masuk poltekkes pasrah sadar diri ga masu...,Awal masuk poltekkes pasrah sadar diri masuk b...,awal masuk poltekkes pasrah sadar diri masuk b...,"[awal, masuk, poltekkes, pasrah, sadar, diri, ...","[awal, masuk, poltekkes, pasrah, sadar, diri, ...","[masuk, poltekkes, pasrah, sadar, masuk, disma..."


In [ ]:
list_stopwords = stopwords.words('indonesian')
list_stopwords.extend(['loh', 'oiya', 'nih', 'ok', 'ah', 'nya', 'banget', 'gue', 'kalo', 'deh', 'pas','gua', 'gw', 'biar', 'iya', 'wkwk', 'hehe', 'kak', 'bisa', 'enggak', 'kayak', 'sih'])
txt_stopword = pd.read_csv('/content/drive/MyDrive/AS/stopwords.txt', names= ["stopwords"], header = None)
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))
list_stopwords = set(list_stopwords)

def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords ]
df['SW Removal'] = df['SW Removal'].apply(stopwords_removal)
df.head()

,Datetime,Username,Tweet,Cleaning,Case Folding,Tokenization,Normalisasi,SW Removal
0,2021-11-01 23:56:14+00:00,viligerrystory,ternyata kuliah msh online jg pdhl katanya off...,ternyata kuliah msh online pdhl katanya offline,ternyata kuliah msh online pdhl katanya offline,"[ternyata, kuliah, msh, online, pdhl, katanya,...","[ternyata, kuliah, masih, online, padahal, kat...","[kuliah, online, offline]"
1,2021-11-01 23:45:26+00:00,Fiayellowluv18,Gpp sihh hujan terus tapi jangan sama petir to...,Gpp sihh hujan terus tapi jangan sama petir to...,gpp sihh hujan terus tapi jangan sama petir to...,"[gpp, sihh, hujan, terus, tapi, jangan, sama, ...","[enggak apa-apa, sih, hujan, terus, tapi, jang...","[hujan, petir, tolong, takut, kadang, mikir, h..."
2,2021-11-01 23:18:49+00:00,sliwerr,"@ciebing sekolahh, tapi online hehe. kemarin g...",sekolahh tapi online hehe kemarin gimanaa kuli...,sekolahh tapi online hehe kemarin gimanaa kuli...,"[sekolahh, tapi, online, hehe, kemarin, gimana...","[sekolahh, tapi, online, hehe, kemarin, bagaim...","[sekolahh, online, kemarin, kuliah, offline, ]"
3,2021-11-01 23:08:21+00:00,deviiiayu96,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR? BIS...,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR BISA...,bisnis sambil sekolah kuliah atau ngantor bisa...,"[bisnis, sambil, sekolah, kuliah, atau, nganto...","[bisnis, sambil, sekolah, kuliah, atau, nganto...","[bisnis, sekolah, kuliah, ngantor, online, sho..."
4,2021-11-01 22:17:08+00:00,Adeliaprbw,Awal masuk poltekkes pasrah sadar diri ga masu...,Awal masuk poltekkes pasrah sadar diri masuk b...,awal masuk poltekkes pasrah sadar diri masuk b...,"[awal, masuk, poltekkes, pasrah, sadar, diri, ...","[awal, masuk, poltekkes, pasrah, sadar, diri, ...","[masuk, poltekkes, pasrah, sadar, masuk, disma..."


In [ ]:
#Stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
def stemming(text):    
    text = [stemmer.stem(word) for word in text]
    return text
df['Stemming'] = df['SW Removal'].apply(stemming)
df.head()

,Datetime,Username,Tweet,Cleaning,Case Folding,Tokenization,Normalisasi,SW Removal,Stemming
0,2021-11-01 23:56:14+00:00,viligerrystory,ternyata kuliah msh online jg pdhl katanya off...,ternyata kuliah msh online pdhl katanya offline,ternyata kuliah msh online pdhl katanya offline,"[ternyata, kuliah, msh, online, pdhl, katanya,...","[ternyata, kuliah, masih, online, padahal, kat...","[kuliah, online, offline]","[kuliah, online, offline]"
1,2021-11-01 23:45:26+00:00,Fiayellowluv18,Gpp sihh hujan terus tapi jangan sama petir to...,Gpp sihh hujan terus tapi jangan sama petir to...,gpp sihh hujan terus tapi jangan sama petir to...,"[gpp, sihh, hujan, terus, tapi, jangan, sama, ...","[enggak apa-apa, sih, hujan, terus, tapi, jang...","[hujan, petir, tolong, takut, kadang, mikir, h...","[hujan, petir, tolong, takut, kadang, mikir, h..."
2,2021-11-01 23:18:49+00:00,sliwerr,"@ciebing sekolahh, tapi online hehe. kemarin g...",sekolahh tapi online hehe kemarin gimanaa kuli...,sekolahh tapi online hehe kemarin gimanaa kuli...,"[sekolahh, tapi, online, hehe, kemarin, gimana...","[sekolahh, tapi, online, hehe, kemarin, bagaim...","[sekolahh, online, kemarin, kuliah, offline, ]","[sekolahh, online, kemarin, kuliah, offline, ]"
3,2021-11-01 23:08:21+00:00,deviiiayu96,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR? BIS...,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR BISA...,bisnis sambil sekolah kuliah atau ngantor bisa...,"[bisnis, sambil, sekolah, kuliah, atau, nganto...","[bisnis, sambil, sekolah, kuliah, atau, nganto...","[bisnis, sekolah, kuliah, ngantor, online, sho...","[bisnis, sekolah, kuliah, ngantor, online, sho..."
4,2021-11-01 22:17:08+00:00,Adeliaprbw,Awal masuk poltekkes pasrah sadar diri ga masu...,Awal masuk poltekkes pasrah sadar diri masuk b...,awal masuk poltekkes pasrah sadar diri masuk b...,"[awal, masuk, poltekkes, pasrah, sadar, diri, ...","[awal, masuk, poltekkes, pasrah, sadar, diri, ...","[masuk, poltekkes, pasrah, sadar, masuk, disma...","[masuk, poltekkes, pasrah, sadar, masuk, disma..."


In [ ]:
import ast
def join_text_list(texts):
    return ' '.join([text for text in texts])

df['Preprocessed'] = df['Stemming'].apply(join_text_list)
df.head()

,Datetime,Username,Tweet,Cleaning,Case Folding,Tokenization,Normalisasi,SW Removal,Stemming,Preprocessed
0,2021-11-01 23:56:14+00:00,viligerrystory,ternyata kuliah msh online jg pdhl katanya off...,ternyata kuliah msh online pdhl katanya offline,ternyata kuliah msh online pdhl katanya offline,"[ternyata, kuliah, msh, online, pdhl, katanya,...","[ternyata, kuliah, masih, online, padahal, kat...","[kuliah, online, offline]","[kuliah, online, offline]",kuliah online offline
1,2021-11-01 23:45:26+00:00,Fiayellowluv18,Gpp sihh hujan terus tapi jangan sama petir to...,Gpp sihh hujan terus tapi jangan sama petir to...,gpp sihh hujan terus tapi jangan sama petir to...,"[gpp, sihh, hujan, terus, tapi, jangan, sama, ...","[enggak apa-apa, sih, hujan, terus, tapi, jang...","[hujan, petir, tolong, takut, kadang, mikir, h...","[hujan, petir, tolong, takut, kadang, mikir, h...",hujan petir tolong takut kadang mikir hujan ku...
2,2021-11-01 23:18:49+00:00,sliwerr,"@ciebing sekolahh, tapi online hehe. kemarin g...",sekolahh tapi online hehe kemarin gimanaa kuli...,sekolahh tapi online hehe kemarin gimanaa kuli...,"[sekolahh, tapi, online, hehe, kemarin, gimana...","[sekolahh, tapi, online, hehe, kemarin, bagaim...","[sekolahh, online, kemarin, kuliah, offline, ]","[sekolahh, online, kemarin, kuliah, offline, ]",sekolahh online kemarin kuliah offline
3,2021-11-01 23:08:21+00:00,deviiiayu96,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR? BIS...,BISNIS SAMBIL SEKOLAH KULIAH ATAU NGANTOR BISA...,bisnis sambil sekolah kuliah atau ngantor bisa...,"[bisnis, sambil, sekolah, kuliah, atau, nganto...","[bisnis, sambil, sekolah, kuliah, atau, nganto...","[bisnis, sekolah, kuliah, ngantor, online, sho...","[bisnis, sekolah, kuliah, ngantor, online, sho...",bisnis sekolah kuliah ngantor online shop prod...
4,2021-11-01 22:17:08+00:00,Adeliaprbw,Awal masuk poltekkes pasrah sadar diri ga masu...,Awal masuk poltekkes pasrah sadar diri masuk b...,awal masuk poltekkes pasrah sadar diri masuk b...,"[awal, masuk, poltekkes, pasrah, sadar, diri, ...","[awal, masuk, poltekkes, pasrah, sadar, diri, ...","[masuk, poltekkes, pasrah, sadar, masuk, disma...","[masuk, poltekkes, pasrah, sadar, masuk, disma...",masuk poltekkes pasrah sadar masuk disma ditri...


In [ ]:
df = df.dropna(subset=['Preprocessed'])
df = df.drop_duplicates(subset=['Preprocessed'])

In [ ]:
df.to_csv("preprocessed.csv")